In [0]:
from datetime import datetime, timedelta 
from getpass import getuser as get_user 
from getpass import getpass as get_password 
from os import environ as enviroment_vars 
from os import setpgrp as set_process_group 
from random import random 
from shlex import split as cmd_split 
from string import punctuation 
from subprocess import Popen, PIPE, STDOUT 
import pandas as pd 

In [0]:
from pyspark import SparkContext, HiveContext, SparkConf  
from pyspark.sql import SparkSession 

ss = SparkSession(sc) 
ss 

In [0]:
#Calculate Base Data for Current and Previous YTD received loans to merge with LRR data later

sql = """
select distinct
    SIT.RUNDATE,
    SIT.REPORT_DATE,
    SIT.cmhc_loan_account_nbr,
    SIT.aplcrecd,/*changed*/
    SIT.REC_YEAR,
    sit.rec_month, /*changed*/
    SIT.LENDER_EN, 
    SIT.LENDER_CDE,
    SIT.PARENT_LENDER_EN, 
    SIT.PARENT_LENDER_CODE
    
from sb_rmo_ho_reporting.lender_dashboard_tat_desj as SIT

order by SIT.cmhc_loan_account_nbr
"""

base_df = ss.sql(sql)
print("Base Data row count: ", base_df.count())
print("Current YTD Base Dataframe creation completed at: ", str(datetime.now()))

Base Data row count:  430286
Current YTD Base Dataframe creation completed at:  2024-09-10 15:04:44.727149


In [0]:
base_df.createOrReplaceTempView("base_df")

df1=spark.sql("""
select distinct
sit.*,
--cast(concat(SUBSTRING(emili.aplcrecd,1,4),"-",SUBSTRING(emili.aplcrecd,5,2),"-","01" ) as date) as Rec_month,
case when 
    frisk.MIAPRCD1 = "26" and  frisk.miaprcd2 is null then 1
    else 0
end as LRR_Only_frisk, --GOOD one!!!

max(case when trim(lrr.loan_referral_reason_tcd) in (26) then 1 else 0 end) as LRR_ever, 

lad.business_source_identification_desc_en
--trim(ltd.contact_nm) as Contact_nm /*lender's RU*/

from base_df sit
Left join ds_ins_risk.ins_risk_last emili
    on sit.cmhc_loan_account_nbr=emili.lnacctno
left join ds_ins_risk.ins_risk_first frisk
on emili.lnacctno = frisk.lnacctno

left join cmwdw_insurance.vwdw_loan_account_dim lad
on emili.lnacctno = lad.cmhc_loan_account_nbr
left join cmwdw_insurance.vwdw_lender_dim len
on emili.AVLENDCD = len.lender_cde
left join cmwdw_insurance.vwdw_costable_unit_dim cu
on emili.cucode = cu.costable_unit_cde

  left join cmwdw_insurance.vwdw_loan_transmission_dim ltd 
    on emili.lnacctno = ltd.cmhc_loan_account_nbr 
    left join cmwdw_insurance.tbdw_tgt_autoreferral_transaction_fact ref 
    on ref.loan_transmission_surrkey = ltd.loan_transmission_surrkey
    left join cmwdw_insurance.tbdw_tgt_loan_referral_reason_dim lrr
    on ref.loan_referral_reason_surrkey = lrr.loan_referral_reason_surrkey

left join cmwdw_insurance.vwdw_loan_account_sum las
on lad.loan_account_surrkey = las.loan_account_surrkey 
left join cmwdw_insurance.tbdw_tgt_property_account_dim prop
on emili.lnacctno =  prop.cmhc_loan_account_nbr
left join cmwdw_insurance.vwdw_loan_transmission_sum lts
on las.loan_account_surrkey=lts.loan_account_surrkey

group by all
""")



In [0]:
#display(dftest.filter((df1['cmhc_loan_account_nbr'] == '25695786')))

In [0]:
'''#Merge Lender Requested Referrals in FIRST RISKING and ALL RISKING with Base Data, and save in AIP Sandbox
lrr_df.createOrReplaceTempView("lrr_df")

lrr_sum_df = spark.sql("""
select
    SIT.cmhc_loan_account_nbr,
    SIT.RUNDATE,
    SIT.aplcrecd,
    SIT.REPORT_DATE,
    SIT.REC_YEAR,
    sit.rec_month,
    SIT.LENDER_EN, 
    SIT.LENDER_CDE,
    SIT.PARENT_LENDER_EN, 
    SIT.PARENT_LENDER_CODE,
    count(SIT.cmhc_loan_account_nbr) as Received,
    sum(lrr.referred_ind_frisk) as referred_frisk,
    sum(lrr.lrr_old) as LRR_frisk_old,
    sum(lrr.llr_only_frisk) as LRR_only_frisk,
    sum(lrr_t.LRR_ever) as LRR_ever

from base_df sit
    left join lrr_df lrr 
        on sit.cmhc_loan_account_nbr=lrr.cmhc_loan_account_nbr
    left join lrr2_df lrr_t
        on sit.cmhc_loan_account_nbr=lrr_t.cmhc_loan_account_nbr
group by all
""")
'''

  File <command-174232460761083>, line 1
    '''#Merge Lender Requested Referrals in FIRST RISKING and ALL RISKING with Base Data, and save in AIP Sandbox
    ^
SyntaxError: incomplete input


In [0]:
#lrr_sum_df = ss.sql(sql_lrr_sum)
df1.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.lender_dashboard_LRR3")
#lrr_sum_df.write.mode("overwrite").insertInto("sb_rmo_ho_reporting.lender_dashboard_LRR")

print("FINAL Lender Requested Referral table row count: ", df1.count())
print("FINAL TABLE creation for Lender Requested Referral Summary completed at: ", str(datetime.now()))

FINAL Lender Requested Referral table row count:  430286
FINAL TABLE creation for Lender Requested Referral Summary completed at:  2024-09-10 15:21:52.377500


In [0]:
%sql
describe extended sb_rmo_ho_reporting.lender_dashboard_LRR

col_name,data_type,comment
RUNDATE,date,null
REPORT_DATE,string,null
cmhc_loan_account_nbr,bigint,null
aplcrecd,string,null
REC_YEAR,string,null
rec_month,string,null
LENDER_EN,string,null
LENDER_CDE,string,null
PARENT_LENDER_EN,string,null
PARENT_LENDER_CODE,string,null


In [0]:
# fixes issue with the writing a table LRR. still use LRR3 for now.
df1.write.mode("overwrite") \
    .format("orc") \
    .option("path", "abfss://sandbox@eppd1stcdp01.dfs.core.windows.net/data/sandbox/SB_RMO_HO_Reporting/SB_RMO_HO_Reporting/lender_dashboard_lrr") \
    .saveAsTable("sb_rmo_ho_reporting.lender_dashboard_LRR")